# Simple Automated Agent with Code Interpreter

A simple automated agent with access to code interpreter

In [ ]:
from azure.identity import DefaultAzureCredential
from pathlib import Path
from dotenv import load_dotenv
import os

from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool

load_dotenv()
project_connection_string = os.environ["PROJECT_CONNECTION_STRING"]
project_connection_string

# Create an Azure AI Client from a connection string
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=project_connection_string
)

def get_file_from_message(messages):
    """
    Checks the list of messages for file attachments (image files or file path annotations)
    and downloads them to the current working directory.
    """
    if "data" in messages and isinstance(messages["data"], list):
        for message_item in messages["data"]:
            for content_item in message_item.get("content", []):
                # Handle image files
                if content_item.get("type") == "image_file":
                    file_id = content_item["image_file"]["file_id"] 
                    print(f"[File] Image File ID: {file_id}")
                    file_name = f"{file_id}_image_file.png"
                    project_client.agents.save_file(file_id=file_id, file_name=file_name)
                    print(f"[File] Saved image file to: {Path.cwd() / file_name}")
                # Handle file path annotations (embedded links)
                elif content_item.get("type") == "text":
                    annotations = content_item["text"].get("annotations", [])
                    for annotation in annotations:
                        if annotation.get("type") == "file_path":
                            print("[File] File Path Found:")
                            print(f"  Text: {annotation['text']}")
                            print(f"  File ID: {annotation['file_path']['file_id']}")
                            file_path_name = Path(annotation["text"]).name
                            project_client.agents.save_file(
                                file_id=annotation["file_path"]["file_id"],
                                file_name=file_path_name
                            )
                            print(f"[File] Saved file to: {Path.cwd() / file_path_name}")


with project_client:
    # Create an instance of the CodeInterpreterTool
    code_interpreter = CodeInterpreterTool()

    # Create the agent
    agent = project_client.agents.create_agent(
        model="gpt-4o-mini",
        name="my-agent",
        instructions="You are helpful agent",
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )

    # Create a thread and add a message
    thread = project_client.agents.create_thread()
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Could you please create a bar chart for the operating profit using the following data and provide the file to me? Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, Company D: $1.8 million",
    )


    # Run the agent
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    # Get messages from the thread
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

    # Manually find the last message from the assistant
    last_msg_from_assistant = None
    if "data" in messages and isinstance(messages["data"], list):
        for m in reversed(messages["data"]):
            if m.get("role") == "assistant":
                last_msg_from_assistant = m
                break

    # Print the last assistant message (text content)
    if last_msg_from_assistant:
        for content_item in last_msg_from_assistant.get("content", []):
            if content_item["type"] == "text":
                text_value = content_item["text"]["value"]
                print(f"Last message from assistant: {text_value}")
                
    else:
        print("No messages found from the assistant.")
        
    get_file_from_message(messages)


    # Delete the agent once done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

Run finished with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_cuKFIObb5LqDelBkkdevmbU3', 'object': 'thread.message', 'created_at': 1743692976, 'assistant_id': 'asst_sHGjBlx0jtTvuNEANaxICbBp', 'thread_id': 'thread_CYnQpmjnUX4cJAFei9c1awic', 'run_id': 'run_4VYaQ4tOJUTJS6oPhtbzstvH', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'I have created the bar chart for the operating profit. You can download it using the link below:\n\n[Download Operating Profit Bar Chart](sandbox:/mnt/data/operating_profit_bar_chart.png)', 'annotations': [{'type': 'file_path', 'text': 'sandbox:/mnt/data/operating_profit_bar_chart.png', 'start_index': 136, 'end_index': 184, 'file_path': {'file_id': 'assistant-P5oYHFW2xxwVyvHvnJhV2M'}}]}}], 'attachments': [{'file_id': 'assistant-P5oYHFW2xxwVyvHvnJhV2M', 'tools': [{'type': 'code_interpreter'}]}], 'metadata': {}}, {'id': 'msg_O1bltYsEOa1YkBV400S5fnYr', 'object': 'thread.message', 'created_at': 1743692968, 'assi

# Conversational Agent (for multi-turn Conversations)

In [7]:
from azure.identity import DefaultAzureCredential
from pathlib import Path
from dotenv import load_dotenv
import os

from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool

# Load environment variables from .env
load_dotenv()
project_connection_string = os.environ["PROJECT_CONNECTION_STRING"]

# Create an Azure AI Client from a connection string
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=project_connection_string
)

def get_file_from_message(messages):
    """
    Checks the list of messages for file attachments (image files or file path annotations)
    and downloads them to the current working directory.
    """
    if "data" in messages and isinstance(messages["data"], list):
        for message_item in messages["data"]:
            for content_item in message_item.get("content", []):
                # Handle image files
                if content_item.get("type") == "image_file":
                    file_id = content_item["image_file"]["file_id"] 
                    print(f"[File] Image File ID: {file_id}")
                    file_name = f"{file_id}_image_file.png"
                    project_client.agents.save_file(file_id=file_id, file_name=file_name)
                    print(f"[File] Saved image file to: {Path.cwd() / file_name}")
                # Handle file path annotations (embedded links)
                elif content_item.get("type") == "text":
                    annotations = content_item["text"].get("annotations", [])
                    for annotation in annotations:
                        if annotation.get("type") == "file_path":
                            print("[File] File Path Found:")
                            print(f"  Text: {annotation['text']}")
                            print(f"  File ID: {annotation['file_path']['file_id']}")
                            file_path_name = Path(annotation["text"]).name
                            project_client.agents.save_file(
                                file_id=annotation["file_path"]["file_id"],
                                file_name=file_path_name
                            )
                            print(f"[File] Saved file to: {Path.cwd() / file_path_name}")

with project_client:
    # Create an instance of the CodeInterpreterTool
    code_interpreter = CodeInterpreterTool()

    # Create the agent with minimal instructions and the tool enabled
    agent = project_client.agents.create_agent(
        model="gpt-4o-mini",
        name="my-agent",
        instructions="You are a helpful agent.",
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )
    print(f"Created agent, agent ID: {agent.id}")

    # Create a conversation thread (to be reused for each turn)
    thread = project_client.agents.create_thread()
    print(f"Created conversation thread, thread ID: {thread.id}\n")

    print("Enter your messages below (type 'exit' to quit):\n")
    
    # Interactive conversation loop
    while True:
        user_input = input("User: ").strip()
        if user_input.lower() in ("exit", "quit"):
            break

        # Create a new message from the user
        project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=user_input,
        )

        # Process the run and let the agent respond
        run = project_client.agents.create_and_process_run(
            thread_id=thread.id, agent_id=agent.id
        )

        if run.status == "failed":
            print(f"[Error] Run failed: {run.last_error}")
            continue

        # Retrieve and display the conversation history
        messages = project_client.agents.list_messages(thread_id=thread.id)
        print("\n--- Conversation History ---")
        if "data" in messages and isinstance(messages["data"], list):
            for msg in messages["data"]:
                # Display user messages
                if msg.get("role") == "user":
                    for content_item in msg.get("content", []):
                        if content_item.get("type") == "text":
                            print(f"User: {content_item['text']['value']}")
                # Display assistant messages
                elif msg.get("role") == "assistant":
                    for content_item in msg.get("content", []):
                        if content_item.get("type") == "text":
                            print(f"Agent: {content_item['text']['value']}")
        else:
            print("No conversation messages found.")
        print("----------------------------\n")

        # Check for any file attachments and download them
        get_file_from_message(messages)

    # Delete the agent when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent.")


Created agent, agent ID: asst_FvGd6umRWRaCscvg9ANdm8qn
Created conversation thread, thread ID: thread_c8u0sj68vaGq5O6qr4HDDmtZ

Enter your messages below (type 'exit' to quit):

Deleted agent.
